In [2]:
from langchain_core.messages import HumanMessage, AIMessage
from app.graph import get_graph

In [3]:
app = get_graph()

In [4]:
q = "How has the debt of Asian Paints changed in the last 3 quarters?"
initial_input = {"query": q}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
# for event in app.stream(initial_input, thread, stream_mode="values"):
#     print(event)

app.invoke(initial_input)

IndexError: tuple index out of range

#### TrobleShooting

In [13]:
# from app.chains import get_data_response_chain, get_df_agent
from app.consts import l_llm, run_get_output
import yfinance as yf
from langchain.prompts import ChatPromptTemplate 
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType


In [6]:
state = {}
state["query"] = "How has the debt of Asian Paints changed in the last 3 quarters?"
state['ticker'] = 'ASIANPAINT.NS'
state['required_data'] = ['quarterly_balance_sheet']

In [7]:
state
stock = yf.Ticker(state['ticker'])

l_df = []
for field in state["required_data"]:
    l_df.append(getattr(stock,field))


In [25]:
def get_df_agent(LLM, l_df, system_prompt, query):
    agent = create_pandas_dataframe_agent(
        LLM,
        l_df,
        verbose=False,
        agent_type=AgentType.OPENAI_FUNCTIONS,
        allow_dangerous_code=True
    )
    prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", '{input}'),
    ]
    )

    chain = prompt | agent | run_get_output
    return chain


In [26]:

system_prompt = "You have to give response to the users query, given the following information"

chain = get_df_agent(LLM=l_llm['gpt-4o'], l_df=l_df, system_prompt=system_prompt, query = state["query"])

res = chain.invoke(state['query'])


In [27]:
res

"Over the last 3 quarters, Asian Paints' net debt increased by ₹1,247.2 crore, and the total debt increased by ₹1,892.5 crore."